<a href="https://colab.research.google.com/github/Arul-Vincy-Mary/nerd/blob/main/project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing required library and package
# data visualization
pip install pyLDAvis
import pyLDAvis.gensim_models
import pyLDAvis

#reading the dataset
import numpy as np
import pandas as pd
import email

#data cleaning
! pip install texthero
import texthero as hero
import re

#Tokenization
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
# NLTK Stop words
nltk.download('stopwords')
from nltk.corpus import stopwords
#spacy for lemmatization
import spacy

#model building
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.phrases import Phrases, Phraser


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/emails.csv")

In [ ]:
data.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [ ]:
data.shape

(517401, 2)

In [ ]:
#message
print(data.loc[1]['message'])

Message-ID: <15464986.1075855378456.JavaMail.evans@thyme>
Date: Fri, 4 May 2001 13:51:00 -0700 (PDT)
From: phillip.allen@enron.com
To: john.lavorato@enron.com
Subject: Re:
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: John J Lavorato <John J Lavorato/ENRON@enronXgate@ENRON>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst

Traveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the business plan meetings here then take a trip without any formal business meetings.  I would even try and get some honest opinions on whether a trip is even desired or necessary.

As far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not.  Too often the

Extract headers:

In [ ]:
# now we add those fields into our dataframe
def get_field(field, messages):
    column = []
    for message in messages:
        e = email.message_from_string(message)
        column.append(e.get(field))
    return column  

In [ ]:
data['subject'] = get_field("Subject", data['message'])
data['X-Folder'] = get_field("X-Folder", data['message'])
data['X-From'] = get_field("X-From", data['message'])
data['X-To'] = get_field("X-To", data['message'])
data.head()   

,file,message,subject,X-Folder,X-From,X-To
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,Re:,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,Re: test,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Phillip K Allen,Leah Van Arsdall
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Phillip K Allen,Randall L Gay
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,Re: Hello,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Phillip K Allen,Greg Piper


In [ ]:
sub = pd.DataFrame(data['subject'])

In [ ]:
data
#sub.head(10000)

In [ ]:
#extract body of the message
def body(messages):
    column = []
    for message in messages:
        e = email.message_from_string(message)
        column.append(e.get_payload())
    return column

In [ ]:
data['body'] = body(data['message'])
data.head()

,file,message,subject,X-Folder,X-From,X-To,body
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,Here is our forecast\n\n
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,Re:,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,Traveling to have a business meeting takes the...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,Re: test,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Phillip K Allen,Leah Van Arsdall,test successful. way to go!!!
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Phillip K Allen,Randall L Gay,"Randy,\n\n Can you send me a schedule of the s..."
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,Re: Hello,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Phillip K Allen,Greg Piper,Let's shoot for Tuesday at 11:45.


Look into X-Folder:

In [ ]:
print("number of folders: ", data.shape[0])
print("number of unique folders: ", data['X-Folder'].unique().shape[0])

number of folders:  517401
number of unique folders:  5336


In [ ]:
print(data['X-Folder'][0])

# we only want last folder name
data['X-Folder'][0].split("\\")[-1]

\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail


"'Sent Mail"

In [ ]:
def preprocess_folder(folders):
    column = []
    for folder in folders:
        if (folder is None or folder == ""):
            column.append(np.nan)
        else:
            column.append(folder.split("\\")[-1].lower())
    return column

data['X-Folder'] = preprocess_folder(data['X-Folder'])
data.head()

,file,message,subject,X-Folder,X-From,X-To,body
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,,'sent mail,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,Here is our forecast\n\n
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,Re:,'sent mail,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,Traveling to have a business meeting takes the...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,Re: test,'sent mail,Phillip K Allen,Leah Van Arsdall,test successful. way to go!!!
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,,'sent mail,Phillip K Allen,Randall L Gay,"Randy,\n\n Can you send me a schedule of the s..."
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,Re: Hello,'sent mail,Phillip K Allen,Greg Piper,Let's shoot for Tuesday at 11:45.


In [ ]:
# view some unique folders
data['X-Folder'].unique()[0:40]

array(["'sent mail", 'all documents', 'contacts', 'deleted items',
       'discussion threads', 'inbox', 'notes inbox', 'sent items', 'sent',
       'straw', '2000 conference', 'active international', 'avaya', 'bmc',
       'bridge', 'bristol babcock', 'colleen koenig', 'compaq',
       'computer associates', 'continental airlines', 'cooper cameron',
       'corestaff', 'dell', 'ebs', 'ees', 'enron europe', 'etol', 'fedex',
       'ge', 'hp', 'human resources', "kinko's", 'nepco europe', 'nepco',
       'oec', 'pcc values', 'personal', 'purchasing', 'requisite', 'sap'],
      dtype=object)

In [ ]:
#copy of the dataset
data_1 = data.copy(deep=True)

Drop the following columns: file,message,date,subject,X-From,X-To

In [ ]:
cols_to_drop = ['file','message','subject','X-From','X-To']
data_1.drop(cols_to_drop, axis=1, inplace=True)
data_1.head()
data_1.shape

(517401, 2)

In [ ]:
#taking subset
data_1 = data_1[data_1["X-Folder"]=="'sent mail"]
data_1['X-Folder'].nunique()
data_1.columns
data_1.shape

(30237, 2)

In [ ]:
data_1.head()

,X-Folder,body
0,'sent mail,Here is our forecast\n\n
1,'sent mail,Traveling to have a business meeting takes the...
2,'sent mail,test successful. way to go!!!
3,'sent mail,"Randy,\n\n Can you send me a schedule of the s..."
4,'sent mail,Let's shoot for Tuesday at 11:45.


In [ ]:
df = data_1[:10000]
df.head()

,X-Folder,body
0,'sent mail,Here is our forecast\n\n
1,'sent mail,Traveling to have a business meeting takes the...
2,'sent mail,test successful. way to go!!!
3,'sent mail,"Randy,\n\n Can you send me a schedule of the s..."
4,'sent mail,Let's shoot for Tuesday at 11:45.


In [ ]:
df_1 = df.copy(deep=True)

In [ ]:
df_1.head()

,X-Folder,body
0,'sent mail,Here is our forecast\n\n
1,'sent mail,Traveling to have a business meeting takes the...
2,'sent mail,test successful. way to go!!!
3,'sent mail,"Randy,\n\n Can you send me a schedule of the s..."
4,'sent mail,Let's shoot for Tuesday at 11:45.


Data cleaning

In [ ]:
for i in range(25):
   print("################################################ EMAIL CONTENT NUMBER:",i,"############################################################################")
   print(df_1.body.iloc[i])

################################################ EMAIL CONTENT NUMBER: 0 ############################################################################
Here is our forecast

 
################################################ EMAIL CONTENT NUMBER: 1 ############################################################################
Traveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the business plan meetings here then take a trip without any formal business meetings.  I would even try and get some honest opinions on whether a trip is even desired or necessary.

As far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not.  Too often the presenter speaks and the others are quiet just waiting for their turn.   The meetings might be better if held in a round table discussion format.  

My suggestion fo

In [ ]:
#there are many emails contain From,To,Subject,cc,---forwarded by in the body itself which is really not required

In [ ]:
df.body.str.contains('[- ]*Forwarded by').value_counts()

False    6402
True     3598
Name: body, dtype: int64

In [ ]:
df_1.body.str.contains('[- ]*Forwarded by').value_counts()

False    6402
True     3598
Name: body, dtype: int64

Looking at the forwarded texts:

In [ ]:
def deal_forwarded(row):
    condition = '[- ]*Forwarded by[\S\s]*Subject:[\S\t ]*'                      
    return re.sub(condition, '', row).strip()
df_1['content1'] = df_1.body.map(deal_forwarded)    

In [ ]:
print(df_1.content1.str.contains('[- ]*Forwarded by').value_counts())

False    9966
True       34
Name: content1, dtype: int64


In [ ]:
for email in df_1.content1[df_1.content1.str.contains('[- ]*Forwarded by')]:
    print(email)
    print("############################ END OF EMAIL ################################################################")

---------------------- Forwarded by Phillip K Allen/HOU/ECT on 09/28/2000 
01:09 PM ---------------------------
   
	
	
	From:  Phillip K Allen                           09/28/2000 10:56 AM
	


Liane,

 As we discussed yesterday, I am concerned there may have been an attempt to 
manipulate the  El Paso San Juan monthly index.  It appears that a single 
buyer entered the marketplace on both September 26 and 27 and paid above 
market prices ($4.70-$4.80) for San Juan gas.  At the time of these trades, 
offers for physical gas at significantly (10 to 15 cents) lower prices were 
bypassed in order to establish higher trades to report into the index 
calculation.  Additionally, these trades are out of line with the associated 
financial swaps for San Juan.

 We have compiled a list of financial and physical trades executed from 
September 25 to September 27.  These are the complete list of trades from 
Enron Online (EOL), Enron's direct phone conversations, and three brokerage 
firms (Amere

there is one more pattern:

In [ ]:
def deal_forwarded_patternless(row):
    condition = '[- ]*Forwarded by[\S\s]*[-]+'
    return re.sub(condition, '', row).strip()
df_1['content2'] = df_1.content1.map(deal_forwarded_patternless)    

In [ ]:
print(df_1.content2.str.contains('[- ]Forwarded by').value_counts())

False    10000
Name: content2, dtype: int64


now there is no forwards in the data!. 
Again check the data

In [ ]:
df_1.head()

,X-Folder,body,content1,content2
0,'sent mail,Here is our forecast\n\n,Here is our forecast,Here is our forecast
1,'sent mail,Traveling to have a business meeting takes the...,Traveling to have a business meeting takes the...,Traveling to have a business meeting takes the...
2,'sent mail,test successful. way to go!!!,test successful. way to go!!!,test successful. way to go!!!
3,'sent mail,"Randy,\n\n Can you send me a schedule of the s...","Randy,\n\n Can you send me a schedule of the s...","Randy,\n\n Can you send me a schedule of the s..."
4,'sent mail,Let's shoot for Tuesday at 11:45.,Let's shoot for Tuesday at 11:45.,Let's shoot for Tuesday at 11:45.


In [ ]:
#copy
df_2 = df_1.copy(deep=True)

In [ ]:
#Getting the final content2 out:
emails = pd.DataFrame(df_2['content2'])

In [ ]:
#Removing the emails that are empty:
emails = emails.loc[~(emails.content2=='')]
emails = emails.rename(columns={'content2':'body'}).reset_index(drop=True)

In [ ]:
print(emails.shape)
emails.head()

(9922, 1)


,body
0,Here is our forecast
1,Traveling to have a business meeting takes the...
2,test successful. way to go!!!
3,"Randy,\n\n Can you send me a schedule of the s..."
4,Let's shoot for Tuesday at 11:45.


In [ ]:
emails1 = emails.drop_duplicates(subset='body', keep='first',inplace=False)
emails1.shape

(9631, 1)

In [ ]:
emails2 = emails1.copy(deep=True)

In [ ]:
emails.shape

(9922, 1)

In [ ]:
#converting uppercase into lower case
emails2['body'] = emails2['body'].str.lower()
emails2.head()

,body
0,here is our forecast
1,traveling to have a business meeting takes the...
2,test successful. way to go!!!
3,"randy,\n\n can you send me a schedule of the s..."
4,let's shoot for tuesday at 11:45.


In [ ]:
#converting uppercase into lower case
emails['body'] = emails['body'].str.lower()
emails.head()

,body
0,here is our forecast
1,traveling to have a business meeting takes the...
2,test successful. way to go!!!
3,"randy,\n\n can you send me a schedule of the s..."
4,let's shoot for tuesday at 11:45.


In [ ]:
#remove contracted
def change_contracted_words(body):
  column = []
  Apos_dict={"'s":" is","n't":" not","'m":" am","'ll":" will",
             "'d":" would","'ve":" have","'re":" are"}
  
  for file in emails['body']:
    for key,value in Apos_dict.items():
      if key in file:
        file = file.replace(key,value)
    column.append(file)
  
  return column

In [ ]:
emails['body'] = change_contracted_words(emails['body'])

In [ ]:
#remove contracted
def change_contracted_words(body):
  column = []
  Apos_dict={"'s":" is","n't":" not","'m":" am","'ll":" will",
             "'d":" would","'ve":" have","'re":" are"}
  
  for file in emails2['body']:
    for key,value in Apos_dict.items():
      if key in file:
        file = file.replace(key,value)
    column.append(file)
  
  return column

In [ ]:
emails2['body'] = change_contracted_words(emails2['body'])

In [ ]:
print(emails.shape)
print(emails2.shape)

(9922, 1)
(9631, 1)


In [ ]:
def remove_duplicates(row):
    condition = '[a-zA-Z]+(@|\.)[a-zA-Z.]+'
    return re.sub(condition, '', str(row)).strip()

In [ ]:
emails['body'] = emails.body.map(remove_duplicates)

In [ ]:
! pip install texthero

In [ ]:
import texthero as hero

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
emails['body'] = hero.remove_urls(emails['body'])

In [ ]:
emails2['body'] = hero.remove_urls(emails2['body'])

In [ ]:
emails['body'] = hero.remove_html_tags(emails['body'])

In [ ]:
emails2['body'] = hero.remove_html_tags(emails2['body'])

In [ ]:
emails['body'] = hero.remove_digits(emails['body'])

In [ ]:
emails2['body'] = hero.remove_digits(emails2['body'])

In [ ]:
emails['body'] = hero.remove_punctuation(emails['body'])

In [ ]:
emails2['body'] = hero.remove_punctuation(emails2['body'])

In [ ]:
emails['body'] = hero.remove_brackets(emails['body'])

In [ ]:
emails2['body'] = hero.remove_brackets(emails2['body'])

In [ ]:
emails['body'] = hero.remove_diacritics(emails['body'])

In [ ]:
emails2['body'] = hero.remove_diacritics(emails2['body'])

In [ ]:
emails['body'] = hero.remove_whitespace(emails['body'])

In [ ]:
emails2['body'] = hero.remove_whitespace(emails2['body'])

In [ ]:
emails['body'].isnull().sum()

In [ ]:
emails.shape #without duplicate,with removing duplicate words

(9922, 1)

In [ ]:
emails2.shape #with removing duplicates, without removing duplicate words

(9631, 1)

In [ ]:
emails_1 = emails.copy(deep=True)

In [ ]:
emails_2 = emails2.copy(deep=True)

Model building

In [ ]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.phrases import Phrases, Phraser

In [ ]:
#Tokenization
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
# NLTK Stop words
nltk.download('stopwords')
from nltk.corpus import stopwords
#spacy for lemmatization
import spacy

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
emails_1['nltk_tokens'] = emails_1['body'].map(lambda r: word_tokenize(r))
emails_1

,body,nltk_tokens
0,here is our forecast,"[here, is, our, forecast]"
1,traveling to have a business meeting takes the...,"[traveling, to, have, a, business, meeting, ta..."
2,test successful way to go,"[test, successful, way, to, go]"
3,randy can you send me a schedule of the salary...,"[randy, can, you, send, me, a, schedule, of, t..."
4,let is shoot for tuesday at,"[let, is, shoot, for, tuesday, at]"
...,...,...
9917,rick unfortunately i had to cancel i had a sic...,"[rick, unfortunately, i, had, to, cancel, i, h..."
9918,hi guys here is a spreadsheet with the list of...,"[hi, guys, here, is, a, spreadsheet, with, the..."
9919,i think everyone got the list make sure we are...,"[i, think, everyone, got, the, list, make, sur..."
9920,troy i am out of the office this morning i wil...,"[troy, i, am, out, of, the, office, this, morn..."


In [ ]:
emails_2['nltk_tokens'] = emails_2['body'].map(lambda r: word_tokenize(r))
emails_2

,body,nltk_tokens
0,here is our forecast,"[here, is, our, forecast]"
1,traveling to have a business meeting takes the...,"[traveling, to, have, a, business, meeting, ta..."
2,test successful way to go,"[test, successful, way, to, go]"
3,randy can you send me a schedule of the salary...,"[randy, can, you, send, me, a, schedule, of, t..."
4,let is shoot for tuesday at,"[let, is, shoot, for, tuesday, at]"
...,...,...
9917,rick unfortunately i had to cancel i had a sic...,"[rick, unfortunately, i, had, to, cancel, i, h..."
9918,hi guys here is a spreadsheet with the list of...,"[hi, guys, here, is, a, spreadsheet, with, the..."
9919,i think everyone got the list make sure we are...,"[i, think, everyone, got, the, list, make, sur..."
9920,troy i am out of the office this morning i wil...,"[troy, i, am, out, of, the, office, this, morn..."


In [ ]:
data_words = emails_1['nltk_tokens'].values.tolist()

In [ ]:
data_words_1 = emails_2['nltk_tokens'].values.tolist()

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

In [ ]:
# Build the bigram and trigram models
bigram_1 = gensim.models.Phrases(data_words_1, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram_1 = gensim.models.Phrases(bigram_1[data_words_1], threshold=100)

In [ ]:
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)

In [ ]:
bigram_mod_1 = Phraser(bigram_1)
trigram_mod_1 = Phraser(trigram_1)

In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod_1[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod_1[bigram_mod_1[doc]] for doc in texts]

In [ ]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ','VERB','ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
stop_words = stopwords.words('english')

In [ ]:
# Remove Stop Words
data_stopwords = remove_stopwords(data_words)

In [ ]:
# Remove Stop Words
data_words_nostops_1 = remove_stopwords(data_words_1)

In [ ]:
# Form Bigrams
bigram_words = make_bigrams(data_stopwords)

In [ ]:
# Form Bigrams
data_words_bigrams_1 = make_bigrams(data_words_nostops_1)

In [ ]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en", disable=['parser', 'ner'])

In [ ]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(bigram_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [ ]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized_1 = lemmatization(data_words_bigrams_1, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

In [ ]:
# Create Dictionary
id2word_1 = corpora.Dictionary(data_lemmatized_1)

In [ ]:
#Create Corpus
texts = data_lemmatized

In [ ]:
#Create Corpus
texts_1 = data_lemmatized_1

In [ ]:
#Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
#Term Document Frequency
corpus_1 = [id2word_1.doc2bow(text) for text in texts]

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=10, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)

In [ ]:
lda_model_2 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=10, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=50,
                                            alpha='auto',
                                            per_word_topics=True)

In [ ]:
lda_model_21 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=10,
                                            passes=50)

In [ ]:
# Build LDA model
lda_model_1 = gensim.models.ldamodel.LdaModel(corpus=corpus_1,
                                            id2word=id2word_1,
                                            num_topics=10, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)

In [ ]:
lda_model_11 = gensim.models.ldamodel.LdaModel(corpus=corpus_1,
                                            id2word=id2word_1,
                                            num_topics=10, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=50,
                                            alpha='auto',
                                            per_word_topics=True)

In [ ]:
lda_model_111 = gensim.models.ldamodel.LdaModel(corpus=corpus_1,
                                            id2word=id2word_1,
                                            num_topics=10,
                                            passes=50)

In [ ]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model_11.print_topics())
doc_lda_11 = lda_model_11[corpus]
doc_lda_11

[(0,
  '0.044*"pool" + 0.035*"ca" + 0.032*"move" + 0.031*"file" + 0.028*"create" + '
  '0.027*"correct" + 0.022*"deliver" + 0.019*"item" + 0.019*"believe" + '
  '0.019*"point"'),
 (1,
  '0.024*"ball" + 0.022*"year" + 0.021*"hit" + 0.019*"bad" + 0.019*"draw" + '
  '0.018*"old" + 0.016*"yard" + 0.016*"stuff" + 0.015*"stop" + '
  '0.011*"husband"'),
 (2,
  '0.080*"deal" + 0.051*"gas" + 0.031*"volume" + 0.025*"capacity" + '
  '0.021*"buy" + 0.020*"price" + 0.019*"month" + 0.017*"power" + 0.017*"sell" '
  '+ 0.017*"market"'),
 (3,
  '0.053*"thank" + 0.025*"question" + 0.018*"number" + 0.018*"change" + '
  '0.018*"storage" + 0.017*"need" + 0.017*"follow" + 0.016*"information" + '
  '0.015*"request" + 0.013*"meeting"'),
 (4,
  '0.032*"say" + 0.022*"utility" + 0.016*"nominate" + 0.014*"state" + '
  '0.013*"pass" + 0.012*"nomination" + 0.011*"tax" + 0.011*"senior" + '
  '0.010*"organization" + 0.009*"expire"'),
 (5,
  '0.062*"may" + 0.048*"time" + 0.026*"schedule" + 0.025*"pm" + 0.022*"guy" + '

In [ ]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model_111.print_topics())
doc_lda_111 = lda_model_111[corpus]
doc_lda_111

[(0,
  '0.019*"new" + 0.018*"transwestern" + 0.014*"way" + 0.010*"draw" + '
  '0.010*"subject" + 0.010*"report" + 0.009*"offer" + 0.008*"space" + '
  '0.008*"president" + 0.008*"due"'),
 (1,
  '0.026*"company" + 0.021*"trade" + 0.014*"offer" + 0.012*"product" + '
  '0.012*"buy" + 0.011*"share" + 0.009*"option" + 0.009*"order" + 0.009*"open" '
  '+ 0.008*"trader"'),
 (2,
  '0.041*"meeting" + 0.029*"thank" + 0.020*"schedule" + 0.019*"time" + '
  '0.019*"attend" + 0.015*"team" + 0.015*"date" + 0.013*"question" + '
  '0.013*"call" + 0.012*"would"'),
 (3,
  '0.046*"deal" + 0.024*"contract" + 0.021*"gas" + 0.018*"thank" + '
  '0.017*"volume" + 0.015*"need" + 0.015*"know" + 0.012*"let" + 0.012*"month" '
  '+ 0.012*"day"'),
 (4,
  '0.023*"thank" + 0.021*"request" + 0.013*"information" + 0.012*"send" + '
  '0.012*"receive" + 0.012*"mail" + 0.010*"name" + 0.010*"may" + '
  '0.010*"approval" + 0.010*"attach"'),
 (5,
  '0.012*"market" + 0.012*"would" + 0.011*"power" + 0.010*"gas" + 0.009*"say" '
 

In [ ]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model_2.print_topics())
doc_lda_2 = lda_model_2[corpus]
doc_lda_2

[(0,
  '0.057*"thank" + 0.046*"know" + 0.044*"need" + 0.037*"let" + 0.031*"subject" '
  '+ 0.029*"call" + 0.025*"send" + 0.021*"question" + 0.019*"change" + '
  '0.018*"number"'),
 (1,
  '0.021*"say" + 0.019*"night" + 0.017*"ball" + 0.016*"woman" + 0.015*"hit" + '
  '0.013*"little" + 0.013*"old" + 0.013*"home" + 0.012*"mailto" + '
  '0.012*"drive"'),
 (2,
  '0.067*"contract" + 0.038*"capacity" + 0.029*"rate" + 0.018*"pay" + '
  '0.016*"pipeline" + 0.016*"invoice" + 0.015*"cost" + 0.015*"agreement" + '
  '0.014*"release" + 0.013*"total"'),
 (3,
  '0.057*"company" + 0.040*"production" + 0.034*"man" + 0.019*"employee" + '
  '0.017*"lunch" + 0.014*"pro" + 0.013*"energy" + 0.013*"internet" + '
  '0.012*"evening" + 0.012*"grant"'),
 (4,
  '0.025*"follow" + 0.024*"service" + 0.021*"request" + 0.019*"system" + '
  '0.019*"customer" + 0.019*"meeting" + 0.017*"schedule" + 0.016*"date" + '
  '0.014*"information" + 0.013*"report"'),
 (5,
  '0.169*"deal" + 0.066*"volume" + 0.043*"month" + 0.036*"se

In [ ]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model_21.print_topics())
doc_lda_21 = lda_model_21[corpus]
doc_lda_21

[(0,
  '0.024*"send" + 0.022*"email" + 0.020*"message" + 0.020*"subject" + '
  '0.019*"mail" + 0.017*"may" + 0.016*"request" + 0.015*"receive" + '
  '0.015*"address" + 0.010*"thank"'),
 (1,
  '0.026*"go" + 0.020*"get" + 0.018*"think" + 0.014*"would" + 0.013*"know" + '
  '0.012*"good" + 0.012*"subject" + 0.012*"see" + 0.012*"say" + 0.011*"want"'),
 (2,
  '0.016*"review" + 0.014*"request" + 0.014*"thank" + 0.013*"list" + '
  '0.012*"information" + 0.011*"date" + 0.010*"access" + 0.010*"process" + '
  '0.010*"attach" + 0.010*"use"'),
 (3,
  '0.014*"power" + 0.014*"would" + 0.012*"market" + 0.011*"project" + '
  '0.010*"company" + 0.010*"gas" + 0.009*"price" + 0.009*"cost" + 0.009*"say" '
  '+ 0.009*"ce"'),
 (4,
  '0.014*"game" + 0.013*"team" + 0.010*"play" + 0.010*"year" + 0.008*"run" + '
  '0.007*"season" + 0.006*"last" + 0.006*"player" + 0.006*"man" + 0.006*"use"'),
 (5,
  '0.042*"deal" + 0.022*"contract" + 0.020*"gas" + 0.015*"volume" + '
  '0.015*"thank" + 0.014*"need" + 0.012*"capaci

In [ ]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
doc_lda

[(0,
  '0.059*"thank" + 0.040*"need" + 0.032*"subject" + 0.031*"know" + '
  '0.029*"call" + 0.029*"let" + 0.026*"send" + 0.021*"question" + '
  '0.020*"change" + 0.020*"buy"'),
 (1,
  '0.035*"say" + 0.019*"ball" + 0.018*"night" + 0.017*"little" + 0.016*"woman" '
  '+ 0.016*"hit" + 0.013*"go" + 0.013*"yard" + 0.012*"old" + 0.012*"mailto"'),
 (2,
  '0.070*"contract" + 0.038*"capacity" + 0.030*"rate" + 0.021*"pay" + '
  '0.017*"invoice" + 0.016*"pipeline" + 0.015*"cost" + 0.015*"release" + '
  '0.014*"agreement" + 0.014*"total"'),
 (3,
  '0.058*"company" + 0.041*"production" + 0.037*"man" + 0.018*"employee" + '
  '0.018*"lunch" + 0.016*"pro" + 0.014*"energy" + 0.014*"internet" + '
  '0.013*"evening" + 0.012*"grant"'),
 (4,
  '0.026*"follow" + 0.024*"service" + 0.021*"request" + 0.020*"customer" + '
  '0.019*"system" + 0.017*"meeting" + 0.016*"date" + 0.016*"schedule" + '
  '0.013*"provide" + 0.013*"information"'),
 (5,
  '0.177*"deal" + 0.069*"volume" + 0.038*"sell" + 0.031*"sale" + '
  '

In [ ]:
#from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model_1.print_topics())
doc_lda_1 = lda_model_1[corpus]
doc_lda_1

[(0,
  '0.047*"pool" + 0.037*"ca" + 0.031*"move" + 0.030*"file" + 0.028*"create" + '
  '0.027*"correct" + 0.022*"deliver" + 0.020*"believe" + 0.019*"item" + '
  '0.019*"point"'),
 (1,
  '0.027*"ball" + 0.023*"hit" + 0.021*"bad" + 0.020*"year" + 0.019*"yard" + '
  '0.018*"old" + 0.016*"stuff" + 0.016*"draw" + 0.016*"stop" + '
  '0.013*"husband"'),
 (2,
  '0.083*"deal" + 0.053*"gas" + 0.032*"volume" + 0.025*"capacity" + '
  '0.022*"buy" + 0.020*"price" + 0.018*"month" + 0.018*"sell" + 0.018*"power" '
  '+ 0.017*"company"'),
 (3,
  '0.056*"thank" + 0.024*"question" + 0.018*"change" + 0.018*"need" + '
  '0.018*"storage" + 0.018*"number" + 0.017*"follow" + 0.016*"information" + '
  '0.015*"request" + 0.012*"meeting"'),
 (4,
  '0.032*"say" + 0.023*"utility" + 0.017*"nominate" + 0.014*"pass" + '
  '0.013*"state" + 0.013*"nomination" + 0.012*"senior" + 0.012*"tax" + '
  '0.011*"organization" + 0.009*"expire"'),
 (5,
  '0.064*"may" + 0.042*"time" + 0.027*"pm" + 0.027*"schedule" + 0.023*"guy" + 

In [ ]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
doc_lda

[(0,
  '0.054*"thank" + 0.053*"know" + 0.042*"need" + 0.036*"let" + 0.030*"subject" '
  '+ 0.027*"call" + 0.020*"make" + 0.019*"see" + 0.018*"send" + 0.018*"get"'),
 (1,
  '0.053*"schedule" + 0.051*"meeting" + 0.034*"meter" + 0.031*"balance" + '
  '0.031*"discuss" + 0.030*"plan" + 0.024*"meet" + 0.020*"attend" + '
  '0.019*"item" + 0.018*"hold"'),
 (2,
  '0.082*"deal" + 0.050*"gas" + 0.044*"contract" + 0.031*"volume" + '
  '0.026*"month" + 0.025*"capacity" + 0.022*"ce" + 0.020*"price" + '
  '0.019*"rate" + 0.018*"power"'),
 (3,
  '0.092*"change" + 0.038*"transport" + 0.034*"comment" + 0.026*"marketing" + '
  '0.023*"ee" + 0.022*"ena" + 0.021*"agree" + 0.016*"currently" + 0.015*"bid" '
  '+ 0.013*"post"'),
 (4,
  '0.022*"order" + 0.017*"person" + 0.017*"continue" + 0.016*"move" + '
  '0.013*"issue" + 0.012*"group" + 0.012*"new" + 0.012*"area" + '
  '0.011*"deliver" + 0.011*"work"'),
 (5,
  '0.033*"service" + 0.030*"company" + 0.018*"provide" + 0.017*"agreement" + '
  '0.016*"business" +

In [ ]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
doc_lda

[(0,
  '0.061*"storage" + 0.054*"customer" + 0.038*"order" + 0.029*"charge" + '
  '0.027*"file" + 0.025*"ee" + 0.024*"notice" + 0.024*"fuel" + 0.019*"late" + '
  '0.016*"item"'),
 (1,
  '0.048*"thank" + 0.047*"know" + 0.034*"need" + 0.033*"let" + 0.032*"would" + '
  '0.026*"subject" + 0.024*"call" + 0.022*"may" + 0.019*"question" + '
  '0.019*"make"'),
 (2,
  '0.037*"book" + 0.032*"time" + 0.031*"schedule" + 0.030*"meeting" + '
  '0.022*"group" + 0.019*"plan" + 0.017*"afternoon" + 0.015*"date" + '
  '0.015*"team" + 0.014*"woman"'),
 (3,
  '0.090*"deal" + 0.048*"contract" + 0.045*"gas" + 0.035*"volume" + '
  '0.027*"month" + 0.024*"ce" + 0.023*"change" + 0.019*"rate" + 0.017*"sale" + '
  '0.017*"sell"'),
 (4,
  '0.030*"power" + 0.029*"market" + 0.021*"company" + 0.017*"demand" + '
  '0.015*"gas" + 0.015*"person" + 0.014*"move" + 0.014*"year" + 0.013*"supply" '
  '+ 0.013*"continue"'),
 (5,
  '0.135*"way" + 0.035*"pay" + 0.023*"do" + 0.021*"first" + 0.021*"check" + '
  '0.019*"balance" +

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))


Perplexity:  -7.905786537165682


In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model_2.log_perplexity(corpus))


Perplexity:  -7.837687450407686


In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model_21.log_perplexity(corpus))


Perplexity:  -7.394131997743233


In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model_11.log_perplexity(corpus))


Perplexity:  -10.799611141185338


In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model_111.log_perplexity(corpus))


Perplexity:  -9.48567249920657


In [ ]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.40661377273123467


In [ ]:
# Compute Coherence Score
coherence_model_lda_11 = CoherenceModel(model=lda_model_11, texts=data_lemmatized_1, dictionary=id2word_1, coherence='c_v')
coherence_lda_11 = coherence_model_lda_11.get_coherence()
print('\nCoherence Score: ', coherence_lda_11)


Coherence Score:  0.3610365247783123


In [ ]:
# Compute Coherence Score
coherence_model_lda_111 = CoherenceModel(model=lda_model_111, texts=data_lemmatized_1, dictionary=id2word_1, coherence='c_v')
coherence_lda_111 = coherence_model_lda_111.get_coherence()
print('\nCoherence Score: ', coherence_lda_111)


Coherence Score:  0.44379659593137494


Data vizulaisation:

In [ ]:
import pyLDAvis.gensim_models
import pyLDAvis

In [ ]:
#Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)

In [ ]:
pyLDAvis.display(vis)

In [ ]:
# Select the model and print the topics
optimal_model = model_list[2]
model_topics = optimal_model.show_topics(formatted=False)
optimal_model.print_topics(num_words=10)

[(0,
  '0.058*"give" + 0.052*"make" + 0.049*"work" + 0.029*"guy" + 0.026*"time" + 0.024*"people" + 0.023*"talk" + 0.022*"issue" + 0.021*"good" + 0.020*"move"'),
 (1,
  '0.033*"company" + 0.024*"price" + 0.023*"market" + 0.021*"year" + 0.021*"trade" + 0.015*"energy" + 0.014*"high" + 0.013*"offer" + 0.011*"share" + 0.011*"demand"'),
 (2,
  '0.040*"send" + 0.037*"email" + 0.034*"message" + 0.032*"mail" + 0.021*"call" + 0.020*"copy" + 0.020*"receive" + 0.016*"address" + 0.013*"information" + 0.012*"free"'),
 (3,
  '0.021*"game" + 0.016*"year" + 0.014*"play" + 0.012*"run" + 0.010*"team" + 0.009*"man" + 0.008*"season" + 0.008*"time" + 0.008*"good" + 0.007*"lose"'),
 (4,
  '0.051*"request" + 0.029*"system" + 0.028*"date" + 0.023*"number" + 0.021*"list" + 0.020*"comment" + 0.019*"receive" + 0.019*"contact" + 0.019*"follow" + 0.018*"regard"'),
 (5,
  '0.118*"deal" + 0.045*"gas" + 0.039*"volume" + 0.034*"contract" + 0.030*"month" + 0.028*"change" + 0.023*"meter" + 0.020*"show" + 0.018*"flow" + 0

In [ ]:
model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(optimal_model)

In [ ]:
#Visualize the topics
pyLDAvis.enable_notebook()
vis_1 = pyLDAvis.gensim_models.prepare(model, corpus,id2word)

In [ ]:
pyLDAvis.display(vis_1)

In [ ]:
# Select the model and print the topics
optimal_model_1 = model_list[2]
model_topics_1 = optimal_model_1.show_topics(formatted=False)
optimal_model_1.print_topics(num_words=10)

[(0,
  '0.075*"meeting" + 0.047*"schedule" + 0.035*"plan" + 0.033*"date" + 0.028*"attend" + 0.028*"time" + 0.025*"discuss" + 0.024*"meet" + 0.019*"group" + 0.016*"question"'),
 (1,
  '0.034*"market" + 0.030*"price" + 0.030*"company" + 0.025*"gas" + 0.024*"power" + 0.018*"trade" + 0.015*"high" + 0.015*"energy" + 0.015*"year" + 0.010*"share"'),
 (2,
  '0.034*"year" + 0.025*"game" + 0.023*"run" + 0.022*"team" + 0.018*"time" + 0.018*"play" + 0.016*"start" + 0.016*"back" + 0.013*"good" + 0.013*"make"'),
 (3,
  '0.046*"request" + 0.043*"review" + 0.030*"process" + 0.022*"follow" + 0.022*"employee" + 0.020*"number" + 0.019*"information" + 0.017*"complete" + 0.016*"approval" + 0.016*"comment"'),
 (4,
  '0.129*"subject" + 0.090*"call" + 0.050*"send" + 0.042*"today" + 0.041*"pm" + 0.035*"give" + 0.034*"cc" + 0.025*"make" + 0.023*"tomorrow" + 0.022*"check"'),
 (5,
  '0.034*"business" + 0.025*"team" + 0.021*"report" + 0.021*"group" + 0.018*"operation" + 0.017*"support" + 0.014*"trading" + 0.014*"r